In [2]:
%load_ext autoreload
%autoreload 2
from data_loader import load_datasets, create_data_loaders
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
from training_pipeline import repeat_training, train_with_different_parameters
from serialization import load
from cnn_model import Net

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
batch_size = 64

In [4]:
train, val, test = load_datasets(32)
train_dat, val_dat, test_dat = create_data_loaders(train, val, test, batch_size)

### Learning rate test

In [ ]:
train_with_different_parameters(5, Net, 10, train_dat, val_dat, test_dat, 'cpu', batch_size, lrs=[i/500 for i in range(1,6)])

### Dropout test

In [ ]:
train_with_different_parameters(5, Net, 10, train_dat, val_dat, test_dat, 'cpu', batch_size, dropouts=[i/10 for i in range(3,8)])

### Betas tests

In [ ]:
train_with_different_parameters(5, Net, 10, train_dat, val_dat, test_dat, 'cpu', batch_size, betas=[(1-i/10, 1-i/1000) for i in range(2,6)])